In [8]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import datetime
import os
import copy
import multiprocessing

writeToFile = True

multithread = True

#This is the current graph file index
currentGraphFile = 0

#This is how many G6 strings go into each file
graphFileSize = 500

#This is the array of Graph6 outputs
outputLineArray = multiprocessing.Queue()

#This is the current output stream
outputStream = None
    
# For a memory cost, we can use this to avoid doing the checks twice for subgraphs when we remove nodes
criticalCache = {}

# This is to test the equality between the Graph6 keys and the str() function keys
testStrCache = {}

#We can also cache the Graph6 strings
graph6Cache = {}

#This is where the g6 files will be stored
imgDirectory = "critical graphs"

def has_valid_coloring(adjacency_dict):

    # graph6DictString = graph_to_graph6_3(adjacency_dict)
    STRdictString = str(adjacency_dict)
    
    # getter = criticalCache.get(graph6DictString, None)
    getterStr = testStrCache.get(STRdictString, None)
    
    if (getterStr != None):
            # print(criticalCache[graph6DictString] == testStrCache[STRdictString])
            return getterStr

    # Determine the maximum degree of any node in the graph
    node_degrees = [len(neighbors) for neighbors in adjacency_dict.values()]
    
    if (len(node_degrees) == 0):
        cache[dictString] = 0
        return 0
    
    max_degree = max(node_degrees)

    # Define a function that recursively tries all possible colorings
    def try_coloring(coloring, node_order):
        # If all nodes have been colored, check if the coloring is valid
        if len(coloring) == len(adjacency_dict):
            for node, neighbors in adjacency_dict.items():
                for neighbor in neighbors:
                    
                    # Normally this check isn't needed, but we need it when we remove nodes from the dictionary
                    # Of course, we need to remove nodes to check if a graph is critical
                    dictNode = coloring.get(node, None)
                    dictNeighbor = coloring.get(neighbor, None)
                    
                    if (dictNode != None and dictNeighbor != None):
                        if dictNode == dictNeighbor:
                            return 0
            
            max_color = max(coloring[node] for node in adjacency_dict.keys())

            # Return the maximum color
            return max_color

        # Otherwise, recursively try all possible color assignments for the next node
        next_node = node_order[len(coloring)]
        neighborhood = adjacency_dict[next_node]
        
        for color in range(max_degree + 1):
            if all(color != coloring.get(neighbor, None) for neighbor in neighborhood):
                coloring[next_node] = color
                
                max_color = try_coloring(coloring, node_order)
                
                if (max_color != 0):
                    # Return the maximum color
                    return max_color
        return 0

    # Generate a list of nodes ordered by degree (highest first)
    nodes = list(adjacency_dict.keys())
    node_order = sorted(nodes, key=lambda node: -len(adjacency_dict[node]))

    # Try all possible colorings starting with an empty coloring
    colors = list(range(max_degree + 1))
    chi = try_coloring({}, node_order)
    # criticalCache[graph6DictString] = chi
    testStrCache[STRdictString] = chi
    # print(cache[dictString])
    return chi + 1

def adjacencyDict(edges):
        # Build a dictionary where each edge is a key and its value is a set of neighboring edges
    adjacency_dict = {}
    for edge in edges:
        if edge[0] not in adjacency_dict:
            adjacency_dict[edge[0]] = set()
        if edge[1] not in adjacency_dict:
            adjacency_dict[edge[1]] = set()
        adjacency_dict[edge[0]].add(edge[1])
        adjacency_dict[edge[1]].add(edge[0])
    
    print(adjacency_dict)
    return adjacency_dict

def remove_node(adj_dict, node):
    # create a copy of the adjacency dictionary to modify
    new_adj_dict = copy.deepcopy(adj_dict)

    # remove the node and its edges from the dictionary
    
    #This takes care of the node
    del new_adj_dict[node]
    
    #This loops through the edges
    for neighbor in adj_dict[node]:
        
        getter = new_adj_dict.get(neighbor, None)
        if (getter != None) :
            if (node in new_adj_dict[neighbor]):
                new_adj_dict[neighbor].remove(node)
    # del new_adj_dict[node]

  # shift down the node indices higher than the removed node
    for i in range(node+1, max(new_adj_dict)+1):
        if i in new_adj_dict:
            new_adj_dict[i-1] = new_adj_dict.pop(i)
            new_adj_dict[i-1] = {j-1 if j>node else j for j in new_adj_dict[i-1]}
  

    return new_adj_dict

#This returns whether a vertex can be deleted while still keeping the chromatic number the same
def isCriticalOnK(adjacency_dict, k):
    
    #We can't have a k-critical graph when there's fewer than k nodes
    #However, we can have a critical graph with k or more nodes depending on where the edges are
    if (len(adjacency_dict) < k):
        return False
    
    #A k-critical graph has exactly k as its chromatic number
    chi = has_valid_coloring(adjacency_dict)
    if (chi != k):
        return False;
    
    
    #For any node that we try to remove, the graph is critical when it loses chromatic number
    for node in adjacency_dict:
        
        #Graphs with isolated vertices are never critical.
        #You can color an isolated vertex whatever you want
        #if (len(adjacency_dict[node]) == 0):
            #print(adjacency_dict)
            #return False
        
        temp_dict = remove_node(adjacency_dict, node)
        chiCrit = has_valid_coloring(temp_dict)
        
        #If the chromatic number stays the same for a node we remove, the graph is not critical
        #Every node must be necessary to keep chi where it is.
        #If there's nodes that aren't strictly necessary, we're hooped
        if (chiCrit == k):
            return False
    
    #The graph has passed all the tests
    return True

def graph_to_graph6_3(adj_dict):
    
    dictString = str(adj_dict)
    
    #This cache can get big, so we only do it for small graphs
    if (len(adj_dict) <= 5):
        getter = graph6Cache.get(dictString, None)
        if (getter != None):
                return getter
    
    # Create a graph object from the adjacency dictionary
    G = nx.Graph(adj_dict)

    # Obtain a Graph6 string from the graph object
    graph6 = nx.to_graph6_bytes(G).decode('ascii')

    # Remove the newline character at the end of the string
    graph6 = graph6.rstrip('\n')
    
    if (len(adj_dict) <= 5):
        graph6Cache[dictString] = graph6
    
    return graph6

def removeSubstring(myStr, substring):
    output_string = ""
    str_list = myStr.split(substring)
    for element in str_list:
        output_string += element
    return output_string

def subdirectory(subdirectoryName, number):
    # Get the current directory path
    current_directory = os.path.dirname(os.path.abspath(__file__))


    # Create the subdirectory if it doesn't exist
    subdirectory_path = os.path.join(current_directory, subdirectoryName)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create the file within the subdirectory
    file_path = os.path.join(subdirectory_path, "classified graphs " + str(number) + ".txt")
    return file_path

def showGraph(dict_):
    #print(dict_)
    
    G = nx.Graph(dict_)
                        
    color_array = ["red", "blue", "green", "gold", "orange", "purple", "teal", "black", "grey", "steelblue", "magenta", "violet", "dodgerblue", "brown"]
    color_array_trunc = []
    
    for i in range(len(dict_)):
        color_array_trunc.append(color_array[i])
        
    # Create a dictionary that maps each node to its color
    # color_map = {node: color_array[coloring[node]] for node in adjacency_dict.keys()}

    # Draw the colored graph using networkx
    nx.draw(G, with_labels=True, node_color=color_array_trunc, font_color='w')
    plt.show()

def classifyGraph(adjacencyDict, minK, maxK):
  
    #We're looking for 
    for i in range(minK, maxK + 1):
        
        if (isCriticalOnK(adjacencyDict, i)):
            # print("Critical graph found!")
            return [True, "The graph is critical on: " + str(i)]
    
    return [False, "The graph is not critical in the k range of " + str(minK) + " - " + str(maxK)]


def multiFoo(graphArrays):
    
    #print(len(graphArrays))
    
    for arr in graphArrays:
        foo(arr)

def foo(inputArray):
        
    global imgDirectory
    global currentGraphFile
    global outputLineArray
    global outputStream

    edges = inputArray[0]
    nodes = inputArray[1]
    
    adj_dict = {node: set() for node in nodes}
    for edge in edges:
        if (len(edge) == 2):
            adj_dict[edge[0]].add(edge[1])
            adj_dict[edge[1]].add(edge[0])


    # This array has True in index 0 if the graph is critical
    # It ha a print message in index 1
    graphResults = classifyGraph(adj_dict, min_k, max_k)
    if (graphResults[0]):

        graph6Str = graph_to_graph6_3(adj_dict)
        graph6Str = removeSubstring(graph6Str, ">>graph6<<")

        global writeToFile
        
        if (writeToFile):
            
            #print(graph6Str)
            outputLineArray.put(graph6Str)
            print(graph6Str)
            

    return

    
def saveFiles():
    
    global writeToFile
    global outputLineArray
    global currentGraphFile
    global outputStream
    
    foundGraphs = 0
    outputLength = 0
    outputTarget = outputLineArray.qsize()
    
    
    print(f"{outputTarget} graphs found!")
    
    if (writeToFile):

        while outputLength != outputTarget:
            
            outputLine = outputLineArray.get()
            
            if (outputLine == None):
                print("No")
                continue
            
            outputStream.write(outputLine)
            outputStream.write("\n")

            foundGraphs = foundGraphs + 1
            outputLength = outputLength + 1
            
            #print(outputLength)
            
            if (foundGraphs > 500):
                foundGraphs = 0
                
                outputStream.close()

                print("File " + str(currentGraphFile) + " Created!")
                currentGraphFile = currentGraphFile + 1

                path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
                outputStream = open(path, "w")
    
        print("File " + str(currentGraphFile) + " Created!")
        outputStream.close()
    

def multithreading2(min_nodes, max_nodes, min_k, max_k, num_threads):
    
    global imgDirectory
    global currentGraphFile
    global outputStream
    
    if (not os.path.exists(imgDirectory)):
        os.makedirs(imgDirectory)
    
    # Create a thread pool executor with the specified number of threads

    print("Multithread 2 enabled")
    
    #Make a 2D array for thread tasks (will become 3D later in code)
    #1st dimension stores the thread to use
    #2nd dimension stores the graph arguments that each thread is in charge of
    #3rd dimension stores the arguments for each graph
    threadTasks = []
    for i in range(num_threads):
        threadTasks.append([])
        
    taskAssign = 0
    
    pool = multiprocessing.Pool()
    
    # Submit each number to the executor for processing

    #futures = []


    path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
    outputStream = open(path, "w")

    allGraphs = 0

    for num_nodes in range(min_nodes, max_nodes+1):
        nauty_generator = graphs.nauty_geng(str(num_nodes) + " -c")
        graphs_list = list(nauty_generator)

        for g in graphs_list:
            #g.show()
            networkx_graph = g.networkx_graph()

            threadTasks[taskAssign].append([networkx_graph.edges(), networkx_graph.nodes()])

            taskAssign = taskAssign + 1
            allGraphs = allGraphs + 1

            if (taskAssign == num_threads):
                taskAssign = 0

            #if multithread:
                #futures.append(executor.submit(foo,[edges, nodes]))
                #executor.submit(foo,[edges, nodes])
            #else:
                #foo([edges, nodes])



    print(f"{allGraphs} graphs searched")
    
    # Get the results from each future as they become available
    pool.map(multiFoo, threadTasks)
    pool.close()
    pool.join()
        
        #results = [future.result() for future in futures]
        
    saveFiles()

    return
                
    
min_nodes = int(input("Enter the min number of nodes: "))
max_nodes = int(input("Enter the max number of nodes: "))

min_k = int(input("Enter the min number of k: "))
max_k = int(input("Enter the max number of k: "))

threads = int(input("Enter the number of threads: "))


print("Starting!")

start_time = datetime.datetime.now()


multithreading2(min_nodes, max_nodes, min_k, max_k, threads)


end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
totalSeconds = elapsed_time.total_seconds()



hours, remainder = divmod(int(totalSeconds), 3600)
minutes, seconds = divmod(remainder, 60)
roundSeconds = float("{:.2f}".format(math.modf(totalSeconds)[0] + seconds))

print("Elapsed time: {} hours, {} minutes, {} seconds".format(int(hours), int(minutes), roundSeconds))

Enter the min number of nodes: 7
Enter the max number of nodes: 8
Enter the min number of k: 5
Enter the max number of k: 6
Enter the number of threads: 4
Starting!
Multithread 2 enabled
FEu~w
G?rL|{
FUZ~wG?rN~{

G?bL]{
G?qk~{
G?qm|{
G?q~~{
G?aM^{
G?z\~{
G?bM\{
GCRS~{G?bL^{

G?bN^{
GCQum{
GCQtn{
GCRUnk
GCQvn{
GCRU|{
GCRvnk
GCRvn{
GCQul{
GCRv~{
GCQtnk
G?qm~[
GCQu~{
G?qm]{11970 graphs searched

GCpU~{
G?~~~{
GCpv^[
GCpu~{
G?z\~[GCpv]{GCrJz{


GCrj~{
GCQUn{
GCRVnk
GCZT}{GCQU~{
GCRU~{

GCQunk
GCQun{
GCQvm{GCZM}{GCY^e{
GCRUl{


GCRVn{GCRu}{

GCY[~{
GCY^n{
GCY^nk
GCQtm{GCY]|{

GCY^~{
GCZvnkGCQv~{

GCZn]{GCRvl{

GCZ]~k
GCRu~{GCZm~{GCpv~{


GCRt~{
GCrJv[
GCZ\~k
GCdfv{GCrJv{

GCde~{
GCpV~{GCfvS{

GCfvU{
GCxu}{
GCxv~{
GCpv^{GCzj~{
GCZU}{

GCpu}{
GCrJu{
GCz^c{
GCz^d{
GCXm~{
GCY^f{
GCvbn{
GCY]l{
GCY[~k
GEhuz{
GCZUm{
GCY]~{
GEht~{
GCZU~{GCZvn{

GCZnZ{
GCZn~{GEvvvs

GEhtv{
GCZM~[GCZ^n{
GEhvv{

GCZ]|{
GEhvn{GEl~vs

GCZ\~{
GEht|{
GCdcu{
GQhVV{GCXnZ{

GQhVv{
GCdc~{
GEh~f{GCY]nk

GCdfvs
GCY]n{
GCdf~{GQj